In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.layers.wrappers import TimeDistributed
import pandas as pd
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using Theano backend.


In [2]:
##seed for random number reproduction
np.random.seed(2017)

In [3]:
##time step addtition to feature 
def dimX(x,ts):
    x=np.asarray(x)
    newX=[]
    for i, c in enumerate(x):
        newX.append([])
        for j in range(ts):
            newX[i].append(c)
    return np.array(newX)

##time step addtition to target
def dimY(Y,ts):
    temp = np.zeros((len(Y), ts, len(chars)), dtype=np.bool)
    for i, c in enumerate(Y):
        for j, s in enumerate(c):
            #print i, j, s
            temp[i, j, char_idx[s]] = 1
    return np.array(temp)

#prediction of argmax
def prediction(preds):
    y_pred=[]
    for i,c in enumerate(preds):
        y_pred.append([])
        for j in c:
            y_pred[i].append(np.argmax(j))
    return np.array(y_pred)

##sequence to text conversion
def seq_txt(y_pred):
    newY=[]
    for i,c in enumerate(y_pred):
        newY.append([])
        for j in c:
            newY[i].append(idx_char[j])
    
    return np.array(newY)
    
##joined smiles output
def smiles_output(s):
    smiles=np.array([])
    for i in s:
        j=''.join(str(k) for k in i)
        smiles=np.append(smiles,j)
    return smiles

In [4]:
##read csv file
data = pd.read_csv('stahl-dataset.csv')
data=data.reindex(np.random.permutation(data.index))

# Extract SMILES Srings and the labelled target vector
Y=data.SMILES
Y.head()
X=data.ix[:,1:7]
X=X.values
X=X.astype('int')
type(X)

numpy.ndarray

In [5]:
print(X)
print(Y.head())

[[1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 ..., 
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]
21     N[S](=O)(=O)C1=CC=C(C=C1)C2=CC(=N[N]2C3=CC=C(F...
30     C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C=C(F)C(=C2)F)C3...
102    C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C(=C)C2)C3=CC=CC=C3
250    CCC(CC)NC1=NC(=CC=N1)C2=C(N=C[N]2C3CC[NH](C)CC...
299         CCOC1=C(Cl)C2=C(C=C(C=C2)[NH+]=C(N)N)C(=O)O1
Name: SMILES, dtype: object


In [6]:
## create mapping of unique SMILES characters to integers
##padding smiles to same length by adding "|" at the end of smiles

maxY=Y.str.len().max()
y=Y.str.ljust(maxY, fillchar='|')
ts=y.str.len().max()

##CharToIndex and IndexToChar functions
chars = sorted(list( set("".join(y.values.flatten()))))
print('total chars:', len(chars))
char_idx= dict((c, i) for i, c in enumerate(chars))
idx_char = dict((i, c) for i, c in enumerate(chars))

y_dash=dimY(y,ts)
x_dash=dimX(X,ts) 

('total chars:', 25)


In [7]:
## Define the LSTM model   
model = Sequential()
model.add(TimeDistributed(Dense(x_dash.shape[1]), input_shape=(x_dash.shape[1],x_dash.shape[2])))
model.add(LSTM(216, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(216, return_sequences=True))
model.add(TimeDistributed(Dense(y_dash.shape[2], activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')

print model.input_shape
print model.output_shape


#model.load_weights("drugai.h5")

(None, 105, 6)
(None, 105, 25)


In [10]:
##For Training uncomment the training section
#start training

model.fit(x_dash,y_dash, nb_epoch=50)

##Epoch 20/20
##335/335 [==============================] - 31s - loss: 1.3319
## Save the model to the checkpoint file 
model.save('molgen.h5')


Epoch 1/50
335/335 [==============================] - 9s - loss: 1.3249     
Epoch 2/50
335/335 [==============================] - 7s - loss: 1.3273     
Epoch 3/50
335/335 [==============================] - 7s - loss: 1.3161     
Epoch 4/50
335/335 [==============================] - 6s - loss: 1.3236     
Epoch 5/50
335/335 [==============================] - 6s - loss: 1.3205     
Epoch 6/50
335/335 [==============================] - 6s - loss: 1.3150     
Epoch 7/50
335/335 [==============================] - 8s - loss: 1.3116     
Epoch 8/50
335/335 [==============================] - 7s - loss: 1.3079     
Epoch 9/50
335/335 [==============================] - 7s - loss: 1.3039     
Epoch 10/50
335/335 [==============================] - 7s - loss: 1.3056     
Epoch 11/50
335/335 [==============================] - 7s - loss: 1.3073     
Epoch 12/50
335/335 [==============================] - 6s - loss: 1.3038     
Epoch 13/50
335/335 [==============================] - 7s - loss: 1.2977 

In [11]:
##For Prediction
## Generate SMILES with the built LSTM model
## basically we feed the model with a pre-defined labelled target vector
## the model will output a string of SMILES characters

#start Prediction
x_pred=[[0,0,0,1,0,0]]
x_pred=dimX(x_pred,ts)      
preds=model.predict(x_pred)
y_pred=prediction(preds)
y_pred=seq_txt(y_pred)


s=smiles_output(y_pred)
print s
#end prediction

[ 'CC(()))CCCCCCCCCCCCCCCCCCCCCCCCCC))||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||']
